In [0]:
import os
import cv2
import pandas as pd
import time
from google.colab import drive

### **Kindly add the Google Drive authentication key in below box so that weights can be stored in it.**

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Data Loading

**Upload the Images data.zip file in the directory for training on these images.**

In [0]:
os.chdir("/content")
!unzip "data.zip"

# Images Preprocessing

**Images Preprocessing perform transformations on images.**

In [0]:
root_dir = os.getcwd()
file_list = ['train.csv', 'val.csv']
image_source_dir = os.path.join(root_dir, 'data/images/')
data_root = os.path.join(root_dir, 'data')
os.chdir(data_root)
for file in file_list:
    flag=0
    print(file)
    image_target_dir = os.path.join(data_root, file.split(".")[0])
    # read list of image files to process from file
    image_list = pd.read_csv(os.path.join(data_root, file), header=0)['image_id']
    print("Start preprocessing images")
    for image in image_list:
        if(image_target_dir.split("/")[len(image_target_dir.split("/"))-1]=="train" and flag==0):
            os.mkdir("train")
            os.chdir(data_root+"/train")
            flag=1
        elif(image_target_dir.split("/")[len(image_target_dir.split("/"))-1]=="val") and flag==0:
            os.chdir(data_root)
            os.mkdir("val")
            os.chdir(data_root+"/val")
            flag=2
        # open image file
        img = cv2.imread(os.path.join(image_source_dir, image))
        print(image_source_dir,image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # perform transformations on image
        b = cv2.distanceTransform(img, distanceType=cv2.DIST_L2, maskSize=5)
        g = cv2.distanceTransform(img, distanceType=cv2.DIST_L1, maskSize=5)
        r = cv2.distanceTransform(img, distanceType=cv2.DIST_C, maskSize=5)
        # merge the transformed channels back to an image
        transformed_image = cv2.merge((b, g, r))
        target_file = os.path.join(image_target_dir, image)
        print("Writing target file {}".format(target_file))
        cv2.imwrite(image, transformed_image)

# Parameters Setting

In [0]:
os.chdir("/content")

In [0]:
!cp data/config.yml config.yml

In [7]:
!pip install luminoth

    100% |████████████████████████████████| 225kB 5.9MB/s 
    100% |████████████████████████████████| 2.3MB 10.8MB/s 
    100% |████████████████████████████████| 71kB 23.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/3f/f0/7c/1ec164f1dc11a6e0ef58463f5e25e83d9c6f661e2e8e70bcef
Successfully built easydict
  Found existing installation: easydict 1.9
    Uninstalling easydict-1.9:
      Successfully uninstalled easydict-1.9
  Found existing installation: Click 7.0
    Uninstalling Click-7.0:
      Successfully uninstalled Click-7.0


In [8]:
!lumi dataset transform --type csv --data-dir data/ --output-dir tfdata/ --split train --split val --only-classes=table


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Saving split "train" in output_dir = tfdata/
  [####################################]  100%             
INFO:tensorflow:Saved 338 records to "tfdata/train.tfrecords"
INFO:tensorflow:Composition per class (train):
INFO:tensorflow:	table: 418
INFO:tensorflow:Saving split "val" in output_dir = tfdata/
  [####################################]  100%
INFO:tensorflow:Saved 65 records to "tfdata/val.tfrecords"
INFO:tensorflow:Composition per class (val):
INFO:tensorflow:	table: 100


In [0]:
os.chdir("/content")

# Tensorboard Visulization

using Tensorboard for visulizing Losses, histograms and projections. Run the below code and open the link. Run the below code of training and refresh the tensorboard page and watch the results of at training iterations. 

In [28]:
from tensorboardcolab import TensorBoardColab
tbc = TensorBoardColab(graph_path='jobs/')
summary_writer = tbc.get_writer()

Wait for 8 seconds...
TensorBoard link:
https://6632d5af.ngrok.io


# Training Start

Epoch set at 10 in config.yml file.

In [29]:
!lumi train -c config.yml


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
Colocations handled automatically by placer.
Instructions f

# Evaluating models

Mean Average Precision (mAP) is the metric commonly used to evaluate object detection task. It computes how well your classifier works across all classes.

What you need to know is that mAP will be a number between 0 and 1, and the higher the better. Moreover, it can be calculated across different IoU (Intersection over Union) thresholds.

For example, Pascal VOC challenge metric uses 0.5 as threshold (notation mAP@0.5), and COCO dataset uses mAP at different thresholds and averages them all out (notation mAP@[0.5:0.95]). Luminoth will print out several of these metrics, specifying the thresholds that were used under this notation.

In Luminoth, lumi eval will make a run through your chosen dataset split (ie. validation or test), and run the model through every image, and then compute metric mAP. Luminoth should now load the last available checkpoint, and start processing images.

You can check it for more details.

[mAP (mean Average Precision) for Object Detection](https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173)




In [30]:
!lumi eval --split val -c config.yml  --no-watch


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
Colocations handled automatically by placer.
Instructions f

# **Creating the Check point  so that it can be tested at running Time.**

In [0]:
checkp_name=!lumi checkpoint create config.yml

In [0]:
checkpoint_name=checkp_name[-1].split(' ')[1]
print (checkpoint_name)

dc7da547920b


#### Enter the checkpoint_name below after export

In [0]:
!lumi checkpoint export dc7da547920b


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Checkpoint dc7da547920b exported successfully.


# **Saving the checkpoint in google drive so that it can be used at testing time.**

Replace the checkpoint name  like this format    (cp checkpoint.tar directory_to_copy/checkpoint.tar)

In [0]:
!cp '72209410e475.tar' '/content/drive/My Drive/72209410e475.tar' 